# Encryption and Decryption

`https://cryptobook.nakov.com/asymmetric-key-ciphers/ecc-encryption-decryption
`

1. pycryptodome : `https://pypi.org/project/pycryptodome/`
2. tinyec : `https://pypi.org/project/tinyec/`

In [1]:
pip install pycryptodome


     |████████████████████████████████| 13.7MB 321kB/s 


In [2]:
pip install tinyec


  Created wheel for tinyec: filename=tinyec-0.3.1-cp36-none-any.whl size=20766 sha256=ef8401aea0269d3d79d535eae67d711a6a5a7feac9acfde892e9c19b4183a90a
  Stored in directory: /root/.cache/pip/wheels/00/2d/bd/610c1d20033b8dfbb4435ece514e2a79e7ad1e8315dae1761f
Successfully built tinyec


In [0]:
from tinyec import registry
# registry holds all the curves of elliptic curve
import secrets
# generating cryptographically strong random numbers suitable for managing data such as passwords, account authentication, security tokens, and related secrets.

In [0]:
from Crypto.Cipher import AES
# https://pycryptodome.readthedocs.io/en/latest/src/cipher/cipher.html
# pycrypto is a collection of both secure hash functions such as SHA256 and various encryption algorithms (AES, DES, RSA, ElGamal, etc.)
# https://pypi.org/project/pycrypto/

In [0]:
import hashlib, binascii
# hashlib used to implement different hash algorithms
# binascii is used to covert between binary and ascii-enocded

AES_GCM: `https://wizardforcel.gitbooks.io/practical-cryptography-for-developers-book/symmetric-key-ciphers/aes-encrypt-decrypt-examples.html`

note:
1. Galois/Counter Mode (GCM) is a mode of operation for symmetric-key cryptographic block ciphers widely adopted for its performance.
2. Counter with CBC-MAC, defined in RFC3610 or NIST SP 800-38C. It only works with ciphers having block size 128 bits (like AES).

It takes input as message and secretkey and outputs the ciphertext, nonce and message authentication code

1. The ciphertext is the encrypted message.
2. The nonce is the randomly generated initial vector (IV) for the GCM construction.
3. The mac is the message authentication code (MAC) calculated during the encryption.

In [0]:
#Encryption and Decryption for AES_GCM
def encrypt_AES_GCM(message, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM)
  # instantiates new GCM cipher object
  cipherText, mac = aesCipher.encrypt_and_digest(message)
  return (cipherText, aesCipher.nonce, mac)


def decrypt_AES_GCM(cipherText, nonce, mac, secretKey):
  aesCipher = AES.new(secretKey, AES.MODE_GCM,nonce)
  # instantiates new GCM cipher object
  plainText = aesCipher.decrypt_and_verify(cipherText, mac)
  return plainText


In [0]:
def ecc_point_to_256_bit_key(point):
  sha = hashlib.sha256(int.to_bytes(point.x, 32, 'big'))
  sha.update(int.to_bytes(point.y, 32, 'big'))
  return sha.digest()

In [0]:
def encrypt_ECC(message, publicKey):
  cipherTextPrivateKey = secrets.randbelow(curve.field.n)
  # creating a private key between 1 and n-1
  sharedECCKey = cipherTextPrivateKey * publicKey
  # cal sharedkey using respective private key and public key
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  # shared key point is hased using ecc_point_to_256_bit_key()
  # result is used as symmetric key for enc/dec
  cipherText, nonce, mac = encrypt_AES_GCM(message, secretKey)
  # encrypt the message using the symmetric key(secretKey)
  cipherTextPublicKey = cipherTextPrivateKey * curve.g
  return (cipherText, nonce, mac, cipherTextPublicKey)


def decrypt_ECC(encryptedMessage, privateKey):
  (cipherText, nonce, mac, cipherTextPublicKey) = encryptedMessage
  sharedECCKey = privateKey * cipherTextPublicKey
  # cal sharedkey using respective private key and public key
  secretKey = ecc_point_to_256_bit_key(sharedECCKey)
  # shared key point is hased using ecc_point_to_256_bit_key()
  # result is used as symmetric key for enc/dec
  plainText = decrypt_AES_GCM(cipherText, nonce, mac, secretKey)
  # decrypt the message using the symmetric key(secretKey)
  return plainText


In [9]:
curve = registry.get_curve('brainpoolP256r1')

message = b'in this pandemic situation they are expecting us to do lab assignments'
privateKey = secrets.randbelow(curve.field.n)
# creating a private key between 1 and n-1
publicKey = privateKey * curve.g
 # cal public key = private key * generator

encryptedMessage = encrypt_ECC(message, publicKey)
encryptedMessageObj =  {
    'cipherText'          : binascii.hexlify(encryptedMessage[0]),
    'nonce'               : binascii.hexlify(encryptedMessage[1]),
    'mac'                 : binascii.hexlify(encryptedMessage[2]),
    'cipherTextPublicKey' : hex(encryptedMessage[3].x) + hex(encryptedMessage[3].y % 2)[2:]
 
}

decryptedMessage = decrypt_ECC(encryptedMessage, privateKey)


print('\n______________________________________ \n')
print('Actual message :',message)
print('Encrypted text:',encryptedMessageObj['cipherText'] )
print('Decrypted text:',decryptedMessage )






______________________________________ 

Actual message : b'in this pandemic situation they are expecting us to do lab assignments'
Encrypted text: b'ea985a2235f8cfb01c0cf7a94abb49d3f39346f0556321bed2e47a6e26e8c66ffdf4541f363dc090f6bad0d05b853a84b5a01ed23eb5a99fa6bd0587c0b40a00616a82924e96'
Decrypted text: b'in this pandemic situation they are expecting us to do lab assignments'


In [10]:
'''print(" *********complete output *******")

print('curve  :', curve)
print('______________________________________________________________________________________________________________________________')
print('a value of curve :', curve.a)
print('b value of curve :',  curve.b)
print('generator point {x, y} :',  curve.field.g)
print('mod value of curve :',  curve.field.n)

print('Private key :',privateKey)
print('Public key point {x, y} :',publicKey)

print('\n______________________________________ \n')
print('Actual message :',message)
print('Encrypted text:',encryptedMessageObj['cipherText'] )
print('Encrypted text:',encryptedMessageObj['nonce'] )
print('Encrypted text:',encryptedMessageObj['mac'] )
print('Decrypted text:',decryptedMessage )'''



'print(" *********complete output *******")\n\nprint(\'curve  :\', curve)\nprint(\'______________________________________________________________________________________________________________________________\')\nprint(\'a value of curve :\', curve.a)\nprint(\'b value of curve :\',  curve.b)\nprint(\'generator point {x, y} :\',  curve.field.g)\nprint(\'mod value of curve :\',  curve.field.n)\n\nprint(\'Private key :\',privateKey)\nprint(\'Public key point {x, y} :\',publicKey)\n\nprint(\'\n______________________________________ \n\')\nprint(\'Actual message :\',message)\nprint(\'Encrypted text:\',encryptedMessageObj[\'cipherText\'] )\nprint(\'Encrypted text:\',encryptedMessageObj[\'nonce\'] )\nprint(\'Encrypted text:\',encryptedMessageObj[\'mac\'] )\nprint(\'Decrypted text:\',decryptedMessage )'